# Juyter notebook Plotting extracts from a belfius bank account

## Data preparation:
Follow the procedure detailled [here](https://www.belfius.be/webapps/fr/selfcare/belfius/comptes/solde-historique/Comment-exporter-mon-historique-vers-un-fichier-CSV-(Excel)-en-Belfius-Direct-Net-) to download the extracts as csv files and put them in a folder named "Data".  

## Dependencies
- jupyter notebook
- pandas
- matplotlib

## Data Loading and cleaning

In [ ]:
import pandas as pd
import os
from datetime import datetime

import matplotlib as mpl
from cycler import cycler
import matplotlib.pyplot as plt
plt.style.use('dark_background')
mpl.rcParams['axes.prop_cycle'] = cycler('color', ['#' + col for col in ['001C7F', '017517', '8C0900', '7600A1', 'B8860B', '006374']])

In [ ]:
DATA = "Data"
ENCODING = 'cp1252'

In [ ]:
data = []
for file in os.listdir(DATA):
    if not file.endswith("csv"):
        continue
    with open(os.path.join(DATA, file), "r", encoding=ENCODING) as reader:
        for i in range(12):  # passing ill-formatted garbage info with balance and balance_date
            line = reader.readline()
            
            # if the balance and balance_date isn't the same in all files, will use the last one silently
            if i == 9:  
                balance = float(line.split(";")[1].split(" ")[0].replace(".", "").replace(",", "."))
            if i == 10:
                balance_date = datetime.strptime(line.split(";")[1][:-1], "%d/%m/%Y %H:%M:%S")
                
        data.append(pd.read_csv(reader, sep=";", thousands='.', decimal=','))

data = pd.concat(data, axis=0)

In [ ]:
# Cases that shouldn't happen
if data.Devise.nunique() != 1:
    print("There are several types of Currencies for different transactions, this will fuck up the accounting")
    
if data.Compte.nunique() != 1:
    print("There are several comptes in the transactions, this will fuck up the accounting")

In [ ]:
col_rename_dict = {
    'Compte': "account", 
    'Date de comptabilisation': "date", 
    "Numéro d'extrait": "extract_num",
    'Numéro de transaction': "transac_number", 
    'Compte contrepartie': "other_account",
    'Nom contrepartie contient': "other_name", 
    'Rue et numéro': "other_street", 
    'Code postal et localité': "other_locality",
    'Transaction': "other_transaction", 
    'Date valeur': "value_date", 
    'Montant': "amount", 
    'Devise': "currency",  
    'Code pays': "country",
    'Communications': "communication"
}

data = data.drop_duplicates()\
    .rename(columns=col_rename_dict)\
    .assign(date=lambda x: pd.to_datetime(x.date, format="%d/%m/%Y"),
            value_date=lambda x: pd.to_datetime(x.value_date, format="%d/%m/%Y"))\
    .sort_values(by="date", ascending=False)\
    .assign(balance=lambda x: balance - x.amount.shift(1, fill_value=0).cumsum())  # the solde after a transaction

## Historic of the amount of money in the account

In [ ]:
xticks = [datetime(year, 1, 1) for year in range(data.date.min().year, data.date.max().year + 2)]
fig, ax = plt.subplots(1, figsize=(14, 8))
ax.set_xticks(xticks)
ax.set_title("Account balance as times passes")
ax.set_xlabel("Date")
ax.set_ylabel("Account Balance (€)")
ax.grid(linestyle="--", linewidth=.5)

plt.plot(data.date, data.balance, linewidth=3);

## Diverse Account statistics

In [ ]:
# transactions where there are no associated account but whose meaning we can mine from other_transaction
index_no_other_name = data.other_name.isna()

index_payment_card = index_no_other_name & data.other_transaction.str.contains("PAIEMENT MAESTRO")
data.loc[index_payment_card, "other_name"] = "card payment"

index_cash_deposit = index_no_other_name & data.other_transaction.str.contains("DEPOT ESPECES AVEC CARTE")
data.loc[index_cash_deposit, "other_name"] = "cash deposit"

index_postal_fees = index_no_other_name & (data.other_transaction.str.contains("FRAIS D'EXPEDITION DE VOS AVIS BANCAIRES") |
                                           data.other_transaction.str.contains("DROIT DE TIMBRE"))
data.loc[index_postal_fees, "other_name"] = "postal fees"

In [ ]:
per_other_stats = data.groupby("other_name").amount.agg(["count", "sum"])

### The 15 entities we gave most money to

In [ ]:
per_other_stats.sort_values("sum").head(15)

### The 15 entities that gave us most money

In [ ]:
per_other_stats.sort_values("sum", ascending=False).head(15)

### The 15 entities we had the most interactions with

In [ ]:
per_other_stats.sort_values("count", ascending=False).head(15)